# Low-Level Design (LLD) of a Hotel Management System

Requirements

- Guests can search for rooms based on room type and price range.
- Guests can book rooms if available.
- Guests can check-in and check-out.
- Hotel staff can assign rooms and manage bookings.
- System should maintain guest details and their booking history.
- System should support multiple payment methods (cash, credit card, online).
- System should support booking cancellation and refund processing.
- System should ensure concurrent booking handling to avoid conflicts.

In [3]:
import threading
import uuid
from collections import defaultdict
from datetime import date, datetime
from enum import Enum

In [4]:
class UserRole(Enum):
    STAFF = 'Staff'
    CUSTOMER = 'Customer'
    ADMIN = 'Admin'

In [5]:
class User:
    
    def __init__(self, name: str, phone: str, email: str, role: UserRole):
        self.user_id = str(uuid.uuid4())
        self.name = name
        self.phone = phone
        self.email = email
        self.role = role
        self.booking_history = []  
        
    def __repr__(self):
        return f"User Name: {self.name}"

In [6]:
class RoomType(Enum):
    SINGLE = 'Single'
    DELUXE = 'Deluxe'
    SUITE = 'Suite'

In [7]:
class Room:
    def __init__(self, room_id: int, room_type: RoomType, price: float):
        self.room_id = room_id
        self.room_type = room_type
        self.price = price
        self.bookings  = []
        self.reserved = []  # Holds rooms temporarily for payment processing
        self.lock = threading.RLock()
        
    def __repr__(self):
        return f"RoomId: {self.room_id} and RoomType: {self.room_type.value}"

    def is_available(self, start_date: date, end_date: date) -> bool:
        """Check if the room is available for booking (excluding reservations)."""
        with self.lock:
            return all(
                end_date <= existing_start or start_date >= existing_end
                for existing_start, existing_end in self.bookings + self.reserved
            )

    def reserve(self, start_date: date, end_date: date) -> bool:
        """Temporarily hold the room for payment processing."""
        with self.lock:
            if self.is_available(start_date, end_date):
                self.reserved.append((start_date, end_date))
                return True
        return False

    def release_hold(self, start_date: date, end_date: date) -> bool:
        """Release the room if payment fails."""
        with self.lock:
            try:
                self.reserved.remove((start_date, end_date))
                return True
            except ValueError:
                return False

    def book(self, start_date: date, end_date: date) -> bool:
        """Confirm the booking after successful payment."""
        with self.lock:
            if (start_date, end_date) in self.reserved:
                self.reserved.remove((start_date, end_date))  # Remove from reserved
                self.bookings.append((start_date, end_date))  # Confirm booking
                return True
        return False

    def free_up(self, start_date: date, end_date: date) -> bool:
        """Cancel a confirmed booking."""
        with self.lock:
            try:
                self.bookings.remove((start_date, end_date))
                return True
            except ValueError:
                return False


In [8]:
class BookingStatus(Enum):
    PENDING = 'Pending'
    CONFIRMED = 'Confirmed'
    CANCELLED = 'Cancelled'
    CHECKED_IN = 'Checked-In'
    CHECKED_OUT = 'Checked-Out'

In [9]:
class Booking:
    
    def __init__(self, user: User, room_type: RoomType, no_of_rooms: int, check_in_time: date, check_out_time: date):
        self.booking_id = str(uuid.uuid4())
        self.user = user
        self.room_type = room_type
        self.no_of_rooms = no_of_rooms
        self.check_in_time = check_in_time
        self.check_out_time = check_out_time
        self.status = BookingStatus.PENDING
        self.booked_rooms = []  # Store assigned rooms
        self.payment = None
        self.lock = threading.RLock()

    def confirm(self, available_rooms):
        """Marks a booking as confirmed and assigns rooms"""
        with self.lock:
            if self.status == BookingStatus.PENDING:
                if len(available_rooms) >= self.no_of_rooms:
                    self.booked_rooms = available_rooms[:self.no_of_rooms]  # Assign rooms
                    for room in self.booked_rooms:
                        room.book(self.check_in_time, self.check_out_time)

                    self.status = BookingStatus.CONFIRMED
                    print(f"✅ Booking {self.booking_id} confirmed for {self.user.name} ({self.room_type.value}, {self.no_of_rooms} rooms) from {self.check_in_time} to {self.check_out_time}.")
                    return True
                print(f"⚠ Not enough available rooms for booking {self.booking_id}.")
        return False
    
    def cancel(self):
        """Cancels a booking and frees up the rooms"""
        with self.lock:
            if self.status in {BookingStatus.PENDING, BookingStatus.CONFIRMED}:
                self.status = BookingStatus.CANCELLED
                
                # Free up rooms
                for room in self.booked_rooms:
                    room.free_up(self.check_in_time, self.check_out_time)
                
                print(f"❌ Booking {self.booking_id} cancelled for {self.user.name}. Rooms released.")
                return True
        print(f"⚠ Booking cancellation failed for {self.user.name}. Status must be PENDING or CONFIRMED.")
        return False
    
    def check_in(self):
        """Marks a booking as checked in"""
        with self.lock:
            if self.status == BookingStatus.CONFIRMED and self.check_in_time <= date.today():
                if self.booked_rooms:
                    self.status = BookingStatus.CHECKED_IN
                    print(f"✅ {self.user.name} has checked in. Assigned rooms: {[room.room_id for room in self.booked_rooms]}")
                    return True
                print(f"⚠ No rooms assigned for booking {self.booking_id}.")
        print(f"⚠ Check-in failed for {self.user.name}. Booking must be CONFIRMED and check-in date should be today.")
        return False

    def check_out(self):
        """Marks a booking as checked out and frees up the rooms"""
        with self.lock:
            if self.status == BookingStatus.CHECKED_IN:
                self.status = BookingStatus.CHECKED_OUT
                
                # Free up rooms after checkout
                for room in self.booked_rooms:
                    room.free_up(self.check_in_time, self.check_out_time)

                print(f"✅ {self.user.name} has checked out. Rooms {', '.join(str(r.room_id) for r in self.booked_rooms)} are now available.")
                return True
        print(f"⚠ Check-out failed for {self.user.name}. Booking must be CHECKED-IN.")
        return False


In [10]:
class PaymentStatus(Enum):
    PENDING = 'Pending'
    COMPLETED = 'Completed'
    FAILED = 'Failed'
    REFUNDED = 'Refunded'

In [11]:
class Payment:
    def __init__(self, booking: Booking, amount: float):
        self.payment_id = str(uuid.uuid4())
        self.booking = booking
        self.amount = amount
        self.status = PaymentStatus.PENDING

    def process_payment(self):
        """Simulate successful payment"""
        self.status = PaymentStatus.COMPLETED
        return True
    
    def refund(self):
        """Refund the payment and cancel booking"""
        if self.status == PaymentStatus.COMPLETED:
            self.status = PaymentStatus.REFUNDED
            return self.amount

In [12]:
class HotelSystem:
    
    _instance = None
    _lock = threading.RLock()
    
    def __new__(cls, name, location):
        if not cls._instance:
            with cls._lock:
                if not cls._instance:
                    cls._instance = super(HotelSystem, cls).__new__(cls)
                    cls._instance.name = name
                    cls._instance.location = location
                    cls._instance.rooms = defaultdict(list)
                    cls._instance.bookings = []
                    cls._instance.users = {}
                    cls._instance.booking_lock = threading.RLock()
        return cls._instance
    
    def add_user(self, user: User):
        self.users[user.user_id] = user
        
    def add_room(self, user: User, room: Room):
        if user.role == UserRole.ADMIN:
            self.rooms[room.room_type].append(room)
        else:
            raise Exception('Unauthorized')

    def remove_room(self, user: User, room: Room):
        if user.role == UserRole.ADMIN:
            if room.room_type in self.rooms:
                self.rooms[room.room_type].remove(room)
        else:
            raise Exception('Unauthorized')

    def search_rooms(self, user: User, room_type: RoomType, min_price: float, max_price: float, start_date: date, end_date: date):
        available_rooms = [
            room for room in self.rooms.get(room_type, [])
            if min_price <= room.price <= max_price
            and room.is_available(start_date, end_date)
        ]
        return available_rooms

    def book_rooms(self, user: User, room_type: RoomType, no_of_rooms: int, check_in_time: date, check_out_time: date):
        """Handles room booking with thread safety"""
        with self.booking_lock:
            available_rooms = [room for room in self.rooms.get(room_type, []) if room.is_available(check_in_time, check_out_time)]

            if len(available_rooms) < no_of_rooms:
                print("⚠ Not enough rooms available for booking.")
                return None

            # Reserve rooms atomically
            total_cost = 0
            booked_rooms = []
            for i in range(no_of_rooms):
                if available_rooms[i].reserve(check_in_time, check_out_time):
                    booked_rooms.append(available_rooms[i])
                    total_cost += available_rooms[i].price

            if len(booked_rooms) == no_of_rooms:
                booking = Booking(user, room_type, no_of_rooms, check_in_time, check_out_time)
                
                # Ask for payment confirmation
                input_value = input('Please confirm payment (yes/no): ')
                if input_value.lower() != 'yes':
                    for room in booked_rooms:
                        room.release_hold(check_in_time, check_out_time)
                    print("⚠ Booking canceled due to payment rejection.")
                    return None

                # Process payment
                booking_payment = Payment(booking, total_cost)
                if not booking_payment.process_payment():
                    for room in booked_rooms:
                        room.release_hold(check_in_time, check_out_time)
                    print("⚠ Booking canceled due to failed payment.")
                    return None

                # Confirm booking after successful payment
                booking.confirm(booked_rooms)
                self.bookings.append(booking)
                booking.payment = booking_payment
                user.booking_history.append(booking)
                print(f"✅ Booking confirmed! {no_of_rooms} {room_type.value} rooms booked for {user.name}.")
                return booking
            else:
                print("⚠ Booking failed due to concurrency issues.")
                return None

        return None
    
    def cancel_booking(self, user: User, booking_id: str):
        """Cancel a booking by ID safely."""
        with self.booking_lock:
            for booking in self.bookings:
                if booking.booking_id == booking_id and booking.user == user:
                    # Process refund
  
                    refunded_amount = booking.payment.refund()
                    
                    booking.cancel()

                    # Remove booking
                    self.bookings.remove(booking)

                    print(f"✅ Booking {booking_id} cancelled. Refunded amount: {refunded_amount}.")
                    return True

        print(f"⚠ Booking {booking_id} not found or already cancelled.")
        return False
    
    def get_user_booking_history(self, user: User):
        if user.booking_history:
            print(f"📜 Booking history for {user.name}:")
            for booking in user.booking_history:
                print(f"{booking.booking_id}- {booking.room_type.value}, {booking.no_of_rooms} rooms from {booking.check_in_time} to {booking.check_out_time} (Status: {booking.status.value})")
        else:
            print(f"⚠ No booking history found for {user.name}.")

In [13]:
class HotelManagementException(Exception):
    pass


class RoomNotAvailableException(HotelManagementException):
    def __init__(self, room_type: RoomType):
        super().__init__(f"No available rooms of type {room_type.value}.")


class RoomNotOccupiedException(HotelManagementException):
    def __init__(self, room_id: int):
        super().__init__(f"Room {room_id} is not occupied.")



In [14]:
if __name__ == "__main__":
    # Initialize system
    hotel = HotelSystem(name="Elite Stay", location="New York")

    # Create users
    admin = User(name="AdminUser", phone="1234567890", email="admin@example.com", role=UserRole.ADMIN)
    customer = User(name="John Doe", phone="9876543210", email="john@example.com", role=UserRole.CUSTOMER)

    hotel.add_user(admin)
    hotel.add_user(customer)
    
    print(hotel.users)

    # Add rooms
    room1 = Room(room_id=101, room_type=RoomType.DELUXE, price=150)
    room2 = Room(room_id=102, room_type=RoomType.DELUXE, price=250)
    hotel.add_room(admin, room1)
    try:
        hotel.add_room(customer, room2)
    except Exception as e:
        print(e)
        hotel.add_room(admin, room2)
        
    print(hotel.rooms)

    # Search
    start_date, end_date = date(2025, 3, 15), date(2025, 3, 20)
    min_price, max_price = 100, 250 
    available_rooms = hotel.search_rooms(customer, RoomType.DELUXE, min_price, max_price, start_date, end_date)
    print(available_rooms)
    # book a room
    
    try:
        booking = hotel.book_rooms(customer, RoomType.DELUXE, 1, start_date, end_date)
    except RoomNotAvailableException as e:
        print(e)

    available_rooms = hotel.search_rooms(customer, RoomType.DELUXE, min_price, max_price, start_date, end_date)
    print(available_rooms)
    
    try:
        booking = hotel.book_rooms(customer, RoomType.DELUXE, 1, start_date, end_date)
    except RoomNotAvailableException as e:
        print(e)
        
    available_rooms = hotel.search_rooms(customer, RoomType.DELUXE, min_price, max_price, start_date, end_date)
    print(available_rooms)
        
    if booking:
        hotel.cancel_booking(customer, booking.booking_id)
        
    available_rooms = hotel.search_rooms(customer, RoomType.DELUXE, min_price, max_price, start_date, end_date)
    print(available_rooms)
    
    # Fetch booking history
    hotel.get_user_booking_history(customer)


{'10c57f71-c00c-485b-b4aa-dbf646e227af': User Name: AdminUser, '03f1306b-d8d9-41a5-94ce-90209c45df87': User Name: John Doe}
Unauthorized
defaultdict(<class 'list'>, {<RoomType.DELUXE: 'Deluxe'>: [RoomId: 101 and RoomType: Deluxe, RoomId: 102 and RoomType: Deluxe]})
[RoomId: 101 and RoomType: Deluxe, RoomId: 102 and RoomType: Deluxe]


Please confirm payment (yes/no):  yes


✅ Booking d7dbc340-0e1d-40f4-a7a0-ca273158b5c0 confirmed for John Doe (Deluxe, 1 rooms) from 2025-03-15 to 2025-03-20.
✅ Booking confirmed! 1 Deluxe rooms booked for John Doe.
[RoomId: 102 and RoomType: Deluxe]


Please confirm payment (yes/no):  yes


✅ Booking ebd50e0c-5cfa-483b-adc2-2c42dc86a671 confirmed for John Doe (Deluxe, 1 rooms) from 2025-03-15 to 2025-03-20.
✅ Booking confirmed! 1 Deluxe rooms booked for John Doe.
[]
❌ Booking ebd50e0c-5cfa-483b-adc2-2c42dc86a671 cancelled for John Doe. Rooms released.
✅ Booking ebd50e0c-5cfa-483b-adc2-2c42dc86a671 cancelled. Refunded amount: 250.
[RoomId: 102 and RoomType: Deluxe]
📜 Booking history for John Doe:
d7dbc340-0e1d-40f4-a7a0-ca273158b5c0- Deluxe, 1 rooms from 2025-03-15 to 2025-03-20 (Status: Confirmed)
ebd50e0c-5cfa-483b-adc2-2c42dc86a671- Deluxe, 1 rooms from 2025-03-15 to 2025-03-20 (Status: Cancelled)
